In [1]:
using CSV
using Dates
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-02-15_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2021-02-09,target_date,hospitalizations
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date,Float64?
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2021-02-05,1001.0,1.0,0.234043,2.82857,2021-02-15,missing
2,11,NORTH BALDWIN INFIRMARY,010129,2021-02-05,1003.0,0.161404,0.1875,1.3636,2021-02-15,missing
3,12,THOMAS HOSPITAL,010100,2021-02-05,1003.0,0.515011,0.203704,4.35101,2021-02-15,missing
4,13,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2021-02-05,1003.0,0.323585,0.55814,2.73378,2021-02-15,missing
5,14,MEDICAL CENTER BARBOUR,010069,2021-02-05,1005.0,1.0,0.333333,1.18964,2021-02-15,missing
6,15,BIBB MEDICAL CENTER,010058,2021-02-05,1007.0,0.821429,0.0,0.556252,2021-02-15,missing
7,16,ST VINCENTS BLOUNT,011305,2021-02-05,1009.0,0.964286,0.4,2.55036,2021-02-15,missing
8,17,BULLOCK COUNTY HOSPITAL,010110,2021-02-05,1011.0,0.0,missing,0.0,2021-02-15,missing
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2021-02-05,1013.0,0.964286,0.466667,0.719357,2021-02-15,missing


In [4]:
function firstval(xs)
    isbad(x) = isnothing(x) || ismissing(x)
    if all(isbad.(xs))
        return missing
    else
        return xs[findfirst(x -> !isbad(x), xs)]
    end
end;

In [5]:
ccn_lookup_df = combine(groupby(rawdata, [:Column1, :hospital_name]), :ccn => firstval => :ccn)
ccn_lookup = Dict(row.Column1 => row.ccn for row in eachrow(ccn_lookup_df));

In [6]:
filter!(row -> ismissing(row["hospitalization_2021-02-09"]), rawdata);

In [7]:
data = select(rawdata,
    :hospital_name => :hospital,
    :Column1 => ByRow(x -> ccn_lookup[x]) => :hospital_id,
    :target_date => :date,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalizations" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-10,0.0,0.932182,0.932182
2,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-11,0.0,0.927556,0.927556
3,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-12,0.0,0.906275,0.906275
4,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-13,0.0,0.882219,0.882219
5,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-14,0.0,0.828106,0.828106
6,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-15,0.0,0.856132,0.856132
7,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-16,0.0,0.81319,0.81319
8,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-17,0.0,0.805505,0.805505
9,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-18,0.0,0.800081,0.800081


In [8]:
data |> CSV.write("../data/hhs_forecast_2021_02_15.csv")

"../data/hhs_forecast_2021_02_15.csv"